In [1]:
import pandas as pd
from collections import Counter
from sklearn.utils import shuffle
from sklearn.metrics.pairwise import pairwise_distances

In [ ]:
# df = pd.read_csv('../data/rating.csv')

In [ ]:
# https://udemy.com/recommender-systems
# https://deeplearningcourses.com/recommender-systems
from __future__ import print_function, division
from builtins import range, input
# Note: you may need to update your version of future
# sudo pip install -U future

import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from datetime import datetime
from sortedcontainers import SortedList

# load in the data
import os
if not os.path.exists('user2movie.json') or \
   not os.path.exists('movie2user.json') or \
   not os.path.exists('usermovie2rating.json') or \
   not os.path.exists('usermovie2rating_test.json'):
   import preprocess2dict


with open('user2movie.json', 'rb') as f:
  user2movie = pickle.load(f)

with open('movie2user.json', 'rb') as f:
  movie2user = pickle.load(f)

with open('usermovie2rating.json', 'rb') as f:
  usermovie2rating = pickle.load(f)

with open('usermovie2rating_test.json', 'rb') as f:
  usermovie2rating_test = pickle.load(f)


N = np.max(list(user2movie.keys())) + 1
# the test set may contain movies the train set doesn't have data on
m1 = np.max(list(movie2user.keys()))
m2 = np.max([m for (u, m), r in usermovie2rating_test.items()])
M = max(m1, m2) + 1
print("N:", N, "M:", M)

if N > 10000:
  print("N =", N, "are you sure you want to continue?")
  print("Comment out these lines if so...")
  exit()


# to find the user similarities, you have to do O(N^2 * M) calculations!
# in the "real-world" you'd want to parallelize this
# note: we really only have to do half the calculations, since w_ij is symmetric
K = 25 # number of neighbors we'd like to consider
limit = 5 # number of common movies users must have in common in order to consider
neighbors = [] # store neighbors in this list
averages = [] # each user's average rating for later use
deviations = [] # each user's deviation for later use

In [ ]:
rating_a = [usermovie2rating[(ii, movie)] for movie in common_movies]
rating_b = [usermovie2rating[(jj, movie)] for movie in common_movies]
ranking_matrix  = pd.DataFrame( [rating_a, rating_b])
ranking_matrix.columns = common_movies
pearson_sim = 1-pairwise_distances(ranking_matrix, metric="correlation")

In [ ]:
def calculate_weight(movie2user, usermovie2rating):
    count = 0
    neighbors = {}  # store neighbors in this list
    K = 25  # number of neighbors we'd like to consider
    limit = 5  # number of common movies users must have in common in order to consider

    for i in movie2user.keys():

        print(i)
        print(100 * (count / len(movie2user)))
        count += 1

        sl = SortedList()

        users_i = movie2user[i]
        users_i_set = set(users_i)

        for j in movie2user.keys():

            if i != j:
                users_j = movie2user[j]
                users_j_set = set(users_j)

                common_users = (users_i_set & users_j_set)  # intersection
                
                if len(common_users) > limit:
                    ratings_i = {user: usermovie2rating[(user, i)] for user in common_users}
                    avg_i = np.mean(list(ratings_i.values()))
                    dev_i = {user: (rating - avg_i) for user, rating in ratings_i.items()}
                    dev_i_values = np.array(list(dev_i.values()))
                    sigma_i = np.sqrt(dev_i_values.dot(dev_i_values))

                    # # save these for later use
                    # averages.append(avg_i)
                    # deviations.append(dev_i)

                    ratings_j = {user: usermovie2rating[(user, j)] for user in common_users}
                    avg_j = np.mean(list(ratings_j.values()))
                    dev_j = {user: (rating - avg_j) for user, rating in ratings_j.items()}
                    dev_j_values = np.array(list(dev_j.values()))
                    sigma_j = np.sqrt(dev_j_values.dot(dev_j_values))

                    numerator = sum(dev_i[m] * dev_j[m] for m in common_users)
                    w_ij = numerator / (sigma_i * sigma_j)

                    sl.add((-w_ij, j))

                    if len(sl) > K:
                        del sl[-1]

                # store the neighbors
                neighbors[i] = sl

    return neighbors

neighbors = calculate_weight(movie2user, usermovie2rating)
neighbors

In [ ]:
neighborns_m = neighbors[41]

for w_ij, j in neighborns_m:
    print(w_ij, j)
    break

In [ ]:
def calculate_rate(i, m, neighbors, usermovie2rating, movie2user):
    neighborns_m = neighbors[m]
    users_m = movie2user[m]
    averages_m = np.mean([usermovie2rating[(user, m)] for user in users_m])
    numerator = 0
    denominator = 0

    for w_ij, j in neighborns_m:

        try:
            users_j = movie2user[j]
            ratings_j = {user: usermovie2rating[(user, j)] for user in users_j}
            avg_j = np.mean(list(ratings_j.values()))

            dev_j = usermovie2rating[(i, j)] - avg_j
            numerator += dev_j * (- w_ij)
            denominator += abs(w_ij)

        except:
            pass

    if denominator == 0:
        prediction = averages_i
    else:
        prediction = numerator / denominator + averages_m
        prediction = min(5, prediction)
        prediction = max(0.5, prediction)  # min rating is 0.5

    return prediction


In [ ]:
neighbors

In [ ]:

calculate_rate(0, 41, neighbors, usermovie2rating, movie2user)

In [ ]:
usermovie2rating[(0, 41)]

In [ ]:
i = 41
j = 232
users_i_set = set(movie2user[i])
users_j_set = set(movie2user[j])
common_users = (users_i_set & users_j_set)  # intersection

ratings_i = {user: usermovie2rating[(user, i)] for user in common_users}
ratings_j = {user: usermovie2rating[(user, j)] for user in common_users}
ranking_matrix = pd.DataFrame([ratings_i, ratings_j], columns = list(common_users))
pearson_sim = 1-pairwise_distances(ranking_matrix, metric="correlation")
pearson_sim

In [ ]:

N = np.max(list(user2movie.keys())) + 1
# the test set may contain movies the train set doesn't have data on
m1 = np.max(list(movie2user.keys()))
m2 = np.max([m for (u, m), r in usermovie2rating_test.items()])
M = max(m1, m2) + 1
print("N:", N, "M:", M)


# initialize variables
K = 10 # latent dimensionality
W = np.random.randn(N, K)
b = np.zeros(N)
U = np.random.randn(M, K)
c = np.zeros(M)
mu = np.mean(list(usermovie2rating.values()))

# prediction[i,j] = W[i].dot(U[j]) + b[i] + c.T[j] + mu

def get_loss(d):
  # d: (user_id, movie_id) -> rating
  N = float(len(d))
  sse = 0
  for k, r in d.items():
    i, j = k
    p = W[i].dot(U[j]) + b[i] + c[j] + mu
    sse += (p - r)*(p - r)
  return sse / N


# train the parameters
epochs = 25
reg =20. # regularization penalty
train_losses = []
test_losses = []
for epoch in range(epochs):
    print("epoch:", epoch)
    epoch_start = datetime.now()
    # perform updates

    # update W and b
    t0 = datetime.now()
    for i in range(N):
    # for W
        matrix = np.eye(K) * reg
        vector = np.zeros(K)
        break
    break


In [ ]:
K = 10 # latent dimensionality
W = np.random.randn(N, K)

In [ ]:
U

In [ ]:
U[[41,1512]]

In [ ]:
U[[41,1512]].T

In [ ]:
U[[41,1512]].dot(W[0])

In [ ]:
W[0].T.dot(U[[41,1512]])

In [ ]:
user2movierating = {}
for i, movies in user2movie.items():
    r = np.array([usermovie2rating[(i,j)] for j in movies])
    user2movierating[i] = (movies, r)
    break

In [ ]:
user2movierating[4943]


In [ ]:
type(movies)

In [ ]:
user2movierating = {}
for i, movies in user2movie.items():
    movies = [int(i) for i in movies]
    r = np.array([usermovie2rating[(i,j)] for j in movies])
    user2movierating[i] = (movies, r)
movie2userrating = {}

In [ ]:
movie2userrating_test = {}

for j, (users, r) in movie2userrating_test.items():
    movie2userrating_test[j][1] = np.array(r)

In [13]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [6]:

docs = ['Well done!',
 'Good work',
 'Great effort',
 'nice work',
 'Excellent!',
 'Weak',
 'Poor effort!',
 'not good',
 'poor work',
 'Could have done better.']
# definir rótulos de clase
labels = np.array([1,1,1,1,1,0,0,0,0,0])

In [7]:

vocab_size = 50
encoded_docs = [one_hot(d, vocab_size) for d in docs]
print(encoded_docs)

[[29, 34], [2, 19], [2, 17], [8, 19], [19], [37], [17, 17], [29, 2], [17, 19], [18, 26, 34, 35]]


In [15]:
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[29 34  0  0]
 [ 2 19  0  0]
 [ 2 17  0  0]
 [ 8 19  0  0]
 [19  0  0  0]
 [37  0  0  0]
 [17 17  0  0]
 [29  2  0  0]
 [17 19  0  0]
 [18 26 34 35]]


In [45]:
max_length

4

In [51]:
model = keras.Sequential()

a =Embedding(vocab_size, 8, input_length=max_length)(0)
a

<tf.Tensor: shape=(8,), dtype=float32, numpy=
array([ 0.02726985,  0.01966386, -0.04280398, -0.01681539, -0.0282212 ,
        0.02146066,  0.02144608,  0.03146023], dtype=float32)>

In [31]:
a

<tf.Tensor: shape=(8,), dtype=float32, numpy=
array([ 0.0102635 , -0.04784817,  0.0404448 , -0.04997382,  0.02917489,
       -0.01177443, -0.03912998, -0.00837523], dtype=float32)>

In [65]:
from tensorflow import keras
from keras.layers import Embedding
from keras.layers import Flatten
from keras.layers import Dense

model = keras.Sequential()
model.add(Embedding(vocab_size, 8, input_length=max_length))
model.add(Flatten())
# model.add(Dense(1, activation='sigmoid'))
# # compilar modelo
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# # resumir modelo
# model.summary()

In [66]:
model.input

<KerasTensor: shape=(None, 4) dtype=float32 (created by layer 'embedding_27_input')>